In [10]:
base="/projects"
pp=base + "/master-theses/henschell"
based=base + "/master-theses/henschell/FastInfantSurfer"
data=base + "/datasets/dHCP/nnUNet_data/nnUNet_raw_data_base/nnUNet_raw_data/Task668_InfantSegmentation/"
#"/datasets/dHCP/nnUNet_data/nnUNet_raw_data_base/Dataset668_InfantSegmentation/"
#
datat1=base + "/datasets/dHCP/nnUNet_data/nnUNet_raw_data_base/nnUNet_raw_data/Task669_InfantSegmentation/"

dataorig=base + "/datasets/dHCP/Data/" # "/datasets/FETA/FastSurfer" #

In [5]:
import pandas as pd
import shutil
from collections import OrderedDict
import json
import sys
sys.path.append("/home/henschell/Test")
sys.path.append("/home/henschell/Test/FastSurfer")
import FastSurfer.FastSurferCNN.data_loader.data_utils as du
import nibabel as nib
import numpy as np

In [6]:
labels = pd.read_csv(based + "/configs/FastInfantSurfer_dHCP_full_LUT.tsv", sep="\t")
labels_full, labels_sag = du.get_labels_from_lut(labels, ("Left-", "ctx-rh"))
lateralization = du.unify_lateralized_labels(labels, ["Left-", "Right-"])
    
labels_dict = pd.Series(labels.LabelName.values, index=labels.MappedID).to_dict()
labels_dict

AttributeError: 'StringMethods' object has no attribute 'removeprefix'

In [ ]:
labels_full

In [7]:
import glob
feta = glob.glob(dataorig + "/sub-*")
feta

['/projects/datasets/FETA/FastSurfer/sub-001',
 '/projects/datasets/FETA/FastSurfer/sub-002',
 '/projects/datasets/FETA/FastSurfer/sub-003',
 '/projects/datasets/FETA/FastSurfer/sub-004',
 '/projects/datasets/FETA/FastSurfer/sub-005',
 '/projects/datasets/FETA/FastSurfer/sub-006',
 '/projects/datasets/FETA/FastSurfer/sub-007',
 '/projects/datasets/FETA/FastSurfer/sub-008',
 '/projects/datasets/FETA/FastSurfer/sub-009',
 '/projects/datasets/FETA/FastSurfer/sub-010',
 '/projects/datasets/FETA/FastSurfer/sub-011',
 '/projects/datasets/FETA/FastSurfer/sub-012',
 '/projects/datasets/FETA/FastSurfer/sub-013',
 '/projects/datasets/FETA/FastSurfer/sub-014',
 '/projects/datasets/FETA/FastSurfer/sub-015',
 '/projects/datasets/FETA/FastSurfer/sub-016',
 '/projects/datasets/FETA/FastSurfer/sub-017',
 '/projects/datasets/FETA/FastSurfer/sub-018',
 '/projects/datasets/FETA/FastSurfer/sub-019',
 '/projects/datasets/FETA/FastSurfer/sub-020',
 '/projects/datasets/FETA/FastSurfer/sub-021',
 '/projects/d

In [15]:
for i in range(len(feta)):
    print(i)
    #subject = feta[i] + "/T2w_orig.nii.gz"
    #seg_new = f"{data}imagesFETA/iseg_{str(i).zfill(3)}_0000.nii.gz"
    #shutil.copyfile(subject, seg_new) #f"{feta[i]}/aseg.nnUnet_full.nii.gz")
    
    # copy labels back
    seg_pred = f"{data}labelsFETA_3D/iseg_{str(i).zfill(3)}.nii.gz"
    new_loc = feta[i] + "/aseg.TestingSet_nnUNet3D_T2_05_T2_full.nii.gz"
    shutil.copyfile(seg_pred, new_loc) #f"{feta[i]}/aseg.nnUnet_full.nii.gz")
    
    seg_pred = f"{data}labelsFETA_2D/iseg_{str(i).zfill(3)}.nii.gz"
    new_loc = feta[i] + "/aseg.TestingSet_nnUNet2D_T2_05_T2_full.nii.gz"
    shutil.copyfile(seg_pred, new_loc) #f"{feta[i]}/aseg.nnUnet_full.nii.gz")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119


In [11]:
# Read training cases
training = pd.read_csv(based + "/data/dataset_split_large_testing_t1t2_meta.tsv", sep="\t")
#training = pd.read_csv("/projects/datasets/dHCP/scripts/sbj_large_testing_t1t2.txt", sep="\t", names=["SubjectFix"])
#training["Resolution"] = 1.0
training.head()

,Path,SubjectFix,scan_age,sex
0,/groups/ag-reuter/projects/datasets/dHCP/Data/...,sub-CC00284AN13_ses-90701,32.14,female
1,/groups/ag-reuter/projects/datasets/dHCP/Data/...,sub-CC00829XX21_ses-17610,33.14,female
2,/groups/ag-reuter/projects/datasets/dHCP/Data/...,sub-CC00136BN13_ses-45000,33.29,female
3,/groups/ag-reuter/projects/datasets/dHCP/Data/...,sub-CC01234AN14_ses-155030,33.29,female
4,/groups/ag-reuter/projects/datasets/dHCP/Data/...,sub-CC00136AN13_ses-45100,33.29,female


In [6]:
def row_handler_intensity(row, img_mod="T2w"):
    sbj = dataorig + row["SubjectFix"] + "/" + img_mod
    if row['Resolution'] == 0.8:
        return sbj + "_08_dhcp.nii.gz"
    elif row['Resolution'] == 0.5:
        return sbj + "_min.nii.gz"
    else:
        return sbj + ".nii.gz"

def row_handler_label(row):
    sbj = dataorig + row["SubjectFix"] + "/" + row["SubjectFix"] +  "_desc-drawem88_dseg" # "_mapped26_dseg" #
    if row['Resolution'] == 0.8:
        return sbj + "_08.nii.gz"
    elif row['Resolution'] == 0.5:
        return sbj + "_min.nii.gz"
    else:
        return sbj + ".nii.gz"
    
training['T2_name'] = training.apply(row_handler_intensity, axis=1)
training['Label_name'] = training.apply(row_handler_label, axis=1)

KeyError: 'Resolution'

In [12]:
#print(training["T2_name"][0])
#print(training["T1_name"][0])
#print(training["Label_name"][0])
print(dataorig)
print(data)
print(datat1)

/projects/datasets/dHCP/Data/
/projects/datasets/dHCP/nnUNet_data/nnUNet_raw_data_base/nnUNet_raw_data/Task668_InfantSegmentation/
/projects/datasets/dHCP/nnUNet_data/nnUNet_raw_data_base/nnUNet_raw_data/Task669_InfantSegmentation/


In [13]:
res_lut = {1.0:  "10", 0.5: "05", 0.8: "08"}
#res_lut[training["Resolution"][1]]

In [14]:
suffix = [".nii.gz", "_min.nii.gz", "_08_dhcp.nii.gz"]
s = ["10", "05", "08"]
res_lut = {1.0:  "10", 0.5: "05", 0.8: "08"}
for i in range(training.shape[0]):
    # cp images with new name to new datapath
    # Copy T1
    #for j in range(3):
    #    num = training.shape[0] * j + i
    #    t1 = training["T1_name"][i] + suffix[j]
    #    t2 = training["T2_name"][i] + suffix[j]
    #    t1w_new = f"{data}imagesTs/iseg_{str(num).zfill(3)}_0000.nii.gz"
    #    shutil.copyfile(t1, t1w_new)
    #shutil.copyfile(f"{feta[i]}/T2w_orig.nii.gz", t1w_new)
    #num = i + 355
    #print(num)
    #t1w_new = f"{datat1}imagesTr/iseg_{str(num).zfill(3)}_0000.nii.gz"
    #seg_new1 = f"{datat1}labelsTr/iseg_{str(num).zfill(3)}.nii.gz"
    # Copy T2
    #t2w_new = f"{data}imagesTr/iseg_{str(num).zfill(3)}_0000.nii.gz"
    #seg_new = f"{data}labelsTr/iseg_{str(num).zfill(3)}.nii.gz"
    #shutil.copyfile(training["T1_name"][i], t1w_new)
    #shutil.copyfile(training["T2_name"][i], t2w_new)
    #shutil.copyfile(training["Label_name"][i], seg_new)
    #shutil.copyfile(training["Label_name"][i], seg_new1)
    #shutil.copyfile(f"{feta[i]}/T2w_orig.nii.gz", t2w_new)
    # Copy labels
    for j in range(3):
        num = training.shape[0] * j + i
        print(num)
    #res_suffix = res_lut[training["Resolution"][i]]
    #for mm in ["FCNN2d", "FCNN3d", "nnUNet2d", "nnUNet3d"]:
        #subject = dataorig + training["SubjectFix"][i] + f"/mri/aseg.TestingSet_nnUNet3D_T2_{s[j]}_T2_full.nii.gz"
        #seg_new = f"{data}labelsTs_3D/iseg_{str(num).zfill(3)}.nii.gz"
        #shutil.copyfile(seg_new, subject) #f"{feta[i]}/aseg.nnUnet_full.nii.gz")
        
        #subject = dataorig + training["SubjectFix"][i] + f"/mri/aseg.TestingSet_nnUNet2D_T2_{s[j]}_T2_full.nii.gz"
        #seg_new = f"{data}labelsTs_2D/iseg_{str(num).zfill(3)}.nii.gz"
        #shutil.copyfile(seg_new, subject) #f"{feta[i]}/aseg.nnUnet_full.nii.gz")
        
        #subject = dataorig + training["SubjectFix"][i] + f"/mri/aseg.TestingSet_nnUNet3D_T1_{s[j]}_T1_full.nii.gz" # f"/T1w{suffix[j]}"  # 
        #seg_new = f"{datat1}labelsTs_3D/iseg_{str(num).zfill(3)}.nii.gz" # f"{datat1}labelsTs/iseg_{str(num).zfill(3)}.nii.gz"
        #shutil.copyfile(seg_new, subject) #f"{feta[i]}/aseg.nnUnet_full.nii.gz")
        
        subject = dataorig + training["SubjectFix"][i] + f"/mri/aseg.TestingSet_nnUNet2D_T1_{s[j]}_T1_full.nii.gz" # f"/T1w{suffix[j]}"  # 
        seg_new = f"{datat1}labelsTs_2D/iseg_{str(num).zfill(3)}.nii.gz" # f"{datat1}labelsTs/iseg_{str(num).zfill(3)}.nii.gz"
        shutil.copyfile(seg_new, subject) #f"{feta[i]}/aseg.nnUnet_full.nii.gz")
    
    #subject = dataorig + training["SubjectFix"][i] + f"/mri/aseg.TestingSet_nnFCNN_T2_{res_suffix}_T2_full.nii.gz"
    #seg_new = f"{data}labelsVal2DCNN/iseg_{str(num).zfill(3)}.nii.gz"
    #shutil.copyfile(seg_new, subject)
        
    #aseg = nib.load(seg_new)
    #aseg_d = np.asanyarray(aseg.dataobj, dtype=np.int16)
    #mapped_aseg, _ = du.map_aparc_aseg2label(aseg_d, aseg_nocc=None,
    #                                         labels=labels_full, labels_sag=labels_sag,
    #                                         sagittal_lut_dict=lateralization
    #                                         )
    #du.save_image(aseg.header, aseg.affine, mapped_aseg, seg_new, dtype=np.int16)
    #shutil.copyfile(training["Label_name"][i], seg_new) 
     # _{s}_T2_full"  
    
    

0
107
214
1
108
215
2
109
216
3
110
217
4
111
218
5
112
219
6
113
220
7
114
221
8
115
222
9
116
223
10
117
224
11
118
225
12
119
226
13
120
227
14
121
228
15
122
229
16
123
230
17
124
231
18
125
232
19
126
233
20
127
234
21
128
235
22
129
236
23
130
237
24
131
238
25
132
239
26
133
240
27
134
241
28
135
242
29
136
243
30
137
244
31
138
245
32
139
246
33
140
247
34
141
248
35
142
249
36
143
250
37
144
251
38
145
252
39
146
253
40
147
254
41
148
255
42
149
256
43
150
257
44
151
258
45
152
259
46
153
260
47
154
261
48
155
262
49
156
263
50
157
264
51
158
265
52
159
266
53
160
267
54
161
268
55
162
269
56
163
270
57
164
271
58
165
272
59
166
273
60
167
274
61
168
275
62
169
276
63
170
277
64
171
278
65
172
279
66
173
280
67
174
281
68
175
282
69
176
283
70
177
284
71
178
285
72
179
286
73
180
287
74
181
288
75
182
289
76
183
290
77
184
291
78
185
292
79
186
293
80
187
294
81
188
295
82
189
296
83
190
297
84
191
298
85
192
299
86
193
300
87
194
301
88
195
302
89
196
303
90
197
304
91
198
30

In [94]:
# Create json file with necessary infos
json_dict = OrderedDict()
json_dict['name'] = "InfantSegmentation"
json_dict['description'] = "dHCP infant segmentation brain - 88 labels"
json_dict['tensorImageSize'] = "4D"
json_dict['reference'] = "see challenge website"
json_dict['licence'] = "see challenge website"
json_dict['release'] = "3.0"
json_dict['modality'] = {
        "0": "T1",
        "1": "T2",
    }
json_dict['labels'] = dict(sorted(labels_dict.items()))
json_dict['numTraining'] = training.shape[0]
json_dict['numTest'] = 0
json_dict['training'] = [[{'image': f"iseg_{str(i).zfill(3)}.nii.gz", 
                           "label": f"iseg_{str(i).zfill(3)}.nii.gz"} for i in
                             range(445)]] #training.shape[0]
json_dict['test'] = []

#save_json(json_dict, os.path.join(out_folder, "dataset.json"))

In [95]:
print(json.dumps(json_dict, indent=4))

{
    "name": "InfantSegmentation",
    "description": "dHCP infant segmentation brain - 88 labels",
    "tensorImageSize": "4D",
    "reference": "see challenge website",
    "licence": "see challenge website",
    "release": "3.0",
    "modality": {
        "0": "T1",
        "1": "T2"
    },
    "labels": {
        "0": "Unknown",
        "1": "Left-Hippocampus",
        "2": "Right-Hippocampus",
        "3": "Left-Amygdala",
        "4": "Right-Amygdala",
        "5": "Left-Cerebral-Cortex",
        "6": "Right-Cerebral-Cortex",
        "7": "Left-Cerebral-White-Matter",
        "8": "Right-Cerebral-White-Matter",
        "9": "Left-Thalamus",
        "10": "Right-Thalamus",
        "11": "Left-Cerebellum",
        "12": "Right-Cerebellum",
        "13": "Brainstem",
        "14": "Right-Caudate_nucleus",
        "15": "Left-Caudate_nucleus",
        "16": "Right-Subthalamic_nucleus",
        "17": "Left-Subthalamic_nucleus",
        "18": "Right-Lentiform_Nucleus",
        "19": "

{0: 'Unknown',
 1: 'Left-Hippocampus',
 2: 'Right-Hippocampus',
 3: 'Left-Amygdala',
 4: 'Right-Amygdala',
 5: 'Left-Cerebral-Cortex',
 6: 'Right-Cerebral-Cortex',
 7: 'Left-Cerebral-White-Matter',
 8: 'Right-Cerebral-White-Matter',
 9: 'Left-Thalamus',
 10: 'Right-Thalamus',
 17: 'Left-Cerebellum',
 18: 'Right-Cerebellum',
 19: 'Brainstem',
 40: 'Right-Caudate_nucleus',
 41: 'Left-Caudate_nucleus',
 44: 'Right-Subthalamic_nucleus',
 45: 'Left-Subthalamic_nucleus',
 46: 'Right-Lentiform_Nucleus',
 47: 'Left-Lentiform_Nucleus',
 48: 'Corpus_Callosum',
 49: 'Left-Lateral-Ventricle',
 50: 'Right-Lateral-Ventricle',
 83: 'CSF',
 84: 'Extra_cranial_background',
 85: 'Right-IntraCranialBackground',
 88: 'Left-IntraCranialBackground'}

In [7]:
# FETA mapping
fdir = "/projects/datasets/FETA/FastSurfer"
# load image
feta_test_gt = nib.load(fdir + "/sub-001/dseg_orig.nii.gz")
gt_d = np.asanyarray(feta_test_gt.dataobj)

# load prediction (full)
feta_test_pred = nib.load(fdir + "/sub-001/aseg.ValidationSet_FastSurferVINN_LatentAug_full.mgz")
pred_d = np.asanyarray(feta_test_pred.dataobj)


In [ ]:
# Get unique labels for ground truth
labels_gt = np.unique(gt_d)
mapping = {}
for lab in labels_gt:
    # Get labels intersecting with gt from pred
    mask = (gt_d == lab)
    mapping[lab] = np.unique(pred_d[mask])